In [3]:
import os
import json
from pathlib import Path

PROFILE_CACHE = {}
base = Path(r"P:\VScode\Billing_Recon\Billing_Recon\profiles")

for partner_dir in base.iterdir():
    if partner_dir.is_dir():
        partner = partner_dir.name

        for version_dir in partner_dir.iterdir():
            if version_dir.is_dir():
                edi_version = version_dir.name
                profile_file = version_dir / "profile.json"

                if profile_file.exists():
                    key = (partner.upper(), edi_version)
                    PROFILE_CACHE[key] = json.loads(profile_file.read_text())
print(f"Loaded {len(PROFILE_CACHE)} partner profiles.")

print(PROFILE_CACHE.keys())

if PROFILE_CACHE.get("GLOBAL", 'Default') is None:
    print("GLOBAL profile not found.")

Loaded 5 partner profiles.
dict_keys([('CARRIER5010', '005010'), ('CARRIERX', '004010'), ('CARRIERY', '004010'), ('CODE', 'Charge_Dictionaries'), ('GLOBAL', 'default')])


In [4]:
import os

edi_text = """ISA*00*          *00*          *ZZ*CARRIERX       *ZZ*OURBROKER     *251101*1430*U*00401*000000001*0*T*>~ 

GS*IN*CARRIERX*OURBROKER*20251101*1430*1*X*004010~ 

ST*210*0001~ 

B3**INV1001*20251101*PP*BOL778231*CC~ 

REF*CN*PR0123456~ 

REF*LO*LD-56712~ 

N1*SH*XYZ CORP~ 

N1*CN*ABC DISTRIBUTION~ 

N1*BT*OURBROKER AP~ 

G62*11*20251030~ 

G62*70*20251102~ 

L1*1*2201.35***BASE FREIGHT~ 

L1*2*192.45***FUEL SURCHARGE~ 

L1*3*150.00***DETENTION~ 

L3*2543.80~ 

SE*15*0001~ 

GE*1*1~ 

IEA*1*000000001~"""

element_delim = '*'
segment_delim = '~'

#remove new line characters
edi_text = edi_text.replace('\n', '')
# print(edi_text)
segments = {}
segments_data = edi_text.strip().split(segment_delim) # for seg in edi_text.split(element_delim) if seg]
for seg in segments_data:
    elements = seg.split(element_delim)
    # print(elements)
    key = elements[0].strip()
    segments.setdefault(key, []).append(elements)
print(segments)
print(type(segments))
required_segments = ['ISA', 'GS', 'ST', 'B3', 'SE']
for req_seg in required_segments:
    if req_seg not in segments:
        print(f"Error: Required segment {req_seg} is missing.")
partner = segments['GS'][0][2].strip()
edi_version = segments['GS'][0][8].strip()
print(f"Partner: {partner}, EDI Version: {edi_version}")

{'ISA': [['ISA', '00', '          ', '00', '          ', 'ZZ', 'CARRIERX       ', 'ZZ', 'OURBROKER     ', '251101', '1430', 'U', '00401', '000000001', '0', 'T', '>']], 'GS': [[' GS', 'IN', 'CARRIERX', 'OURBROKER', '20251101', '1430', '1', 'X', '004010']], 'ST': [[' ST', '210', '0001']], 'B3': [[' B3', '', 'INV1001', '20251101', 'PP', 'BOL778231', 'CC']], 'REF': [[' REF', 'CN', 'PR0123456'], [' REF', 'LO', 'LD-56712']], 'N1': [[' N1', 'SH', 'XYZ CORP'], [' N1', 'CN', 'ABC DISTRIBUTION'], [' N1', 'BT', 'OURBROKER AP']], 'G62': [[' G62', '11', '20251030'], [' G62', '70', '20251102']], 'L1': [[' L1', '1', '2201.35', '', '', 'BASE FREIGHT'], [' L1', '2', '192.45', '', '', 'FUEL SURCHARGE'], [' L1', '3', '150.00', '', '', 'DETENTION']], 'L3': [[' L3', '2543.80']], 'SE': [[' SE', '15', '0001']], 'GE': [[' GE', '1', '1']], 'IEA': [[' IEA', '1', '000000001']], '': [['']]}
<class 'dict'>
Partner: CARRIERX, EDI Version: 004010


In [22]:
profile = PROFILE_CACHE[('GLOBAL', 'default')]
profile = PROFILE_CACHE.get(('CARRIERX', '004010'),[])
if len(profile) == 0:
    profile = PROFILE_CACHE[('GLOBAL', 'default')]

invoice_id_seg = profile['segments']['header']['invoice_id']

print(profile)

print(f"Invoice ID Segment: {invoice_id_seg}")

{'partner': 'CARRIERX', 'edi_version': '004010', 'profile_version': '1.0.0', 'envelope': {'detect': {'gs_sender': True, 'isa_sender': True}}, 'segments': {'header': {'invoice_id': {'seg': 'B3', 'idx': 2}, 'invoice_date': {'seg': 'B3', 'idx': 3, 'fmt': 'yyyyMMdd'}, 'bol': {'firstOf': [{'seg': 'B3', 'idx': 5}, {'seg': 'REF', 'qual': 'BM', 'idx': 2}]}, 'pro': {'seg': 'REF', 'qual': 'CN', 'idx': 2}, 'load_id': {'seg': 'REF', 'qual': 'LO', 'idx': 2}}, 'parties': [{'seg': 'N1', 'qual': 'SH', 'nameIdx': 2, 'mapTo': 'parties.ship_from'}, {'seg': 'N1', 'qual': 'CN', 'nameIdx': 2, 'mapTo': 'parties.ship_to'}, {'seg': 'N1', 'qual': 'BT', 'nameIdx': 2, 'mapTo': 'parties.bill_to'}], 'dates': [{'seg': 'G62', 'qual': '11', 'mapTo': 'dates.pickup', 'fmt': 'yyyyMMdd'}, {'seg': 'G62', 'qual': '70', 'mapTo': 'dates.delivery', 'fmt': 'yyyyMMdd'}], 'charges': {'strategy': 'L1_only', 'l1_rules': [{'contains': ['BASE FREIGHT', 'LINEHAUL'], 'mapTo': 'charges.base_freight'}, {'contains': ['FSC', 'FUEL SURCHARG

In [6]:
header = profile['segments']['header']
print(f"Header Segment: {header}")
parties = profile['segments']['parties']
print(f"Parties Segment: {parties}")
dates = profile['segments']['dates']
print(f"Dates Segment: {dates}")
charges = profile['segments']['charges']
print(f"Charges Segment: {charges}")
total_rules = profile['segments']['total']
print(f"Totals Segment: {total_rules}")
currency_rules = profile['currency']
print(f"Currency Segment: {currency_rules}")
    

Header Segment: {'invoice_id': {'seg': 'B3', 'idx': 2}, 'invoice_date': {'seg': 'B3', 'idx': 3, 'fmt': 'yyyyMMdd'}, 'bol': {'firstOf': [{'seg': 'B3', 'idx': 5}, {'seg': 'REF', 'qual': 'BM', 'idx': 2}]}, 'pro': {'seg': 'REF', 'qual': 'CN', 'idx': 2}}
Parties Segment: [{'seg': 'N1', 'qual': 'SH', 'nameIdx': 2, 'mapTo': 'parties.ship_from'}, {'seg': 'N1', 'qual': 'CN', 'nameIdx': 2, 'mapTo': 'parties.ship_to'}, {'seg': 'N1', 'qual': 'BT', 'nameIdx': 2, 'mapTo': 'parties.bill_to'}]
Dates Segment: [{'seg': 'G62', 'qual': '11', 'mapTo': 'dates.pickup', 'fmt': 'yyyyMMdd'}, {'seg': 'G62', 'qual': '70', 'mapTo': 'dates.delivery', 'fmt': 'yyyyMMdd'}]
Charges Segment: {'strategy': 'L1_only', 'l1_rules': [{'contains': ['BASE FREIGHT', 'LINEHAUL'], 'mapTo': 'charges.base_freight'}, {'contains': ['FSC', 'FUEL SURCHARGE'], 'mapTo': 'charges.fuel_surcharge'}, {'contains': ['DETENTION'], 'mapTo': 'charges.detention'}, {'defaultOther': True}], 'sac_rules': []}
Totals Segment: {'seg': 'L3', 'idx': 1, 'fa

In [7]:
invoice_id_rule = header['invoice_id']
print(f"Invoice ID rule: {invoice_id_rule}")
invoice_date_rule = header['invoice_date']
print(f"Invoice Date rule: {invoice_date_rule}")
bol_rule = header['bol']["firstOf"]
print(f"BOL rule: {bol_rule}")
pro_rule = header['pro']
print(f"PRO rule: {pro_rule}")
header['load_id'] = {'seg': 'REF', 'qual': 'LO', 'idx': 2}
load_id_rule = header['load_id']
print(f"Load ID rule: {load_id_rule}")

Invoice ID rule: {'seg': 'B3', 'idx': 2}
Invoice Date rule: {'seg': 'B3', 'idx': 3, 'fmt': 'yyyyMMdd'}
BOL rule: [{'seg': 'B3', 'idx': 5}, {'seg': 'REF', 'qual': 'BM', 'idx': 2}]
PRO rule: {'seg': 'REF', 'qual': 'CN', 'idx': 2}
Load ID rule: {'seg': 'REF', 'qual': 'LO', 'idx': 2}


In [8]:
for party in parties:
    if party['mapTo'] == "parties.ship_from":
        ship_from_rule = party
        print(f"Ship From rule: {ship_from_rule}")
    elif party['mapTo'] == "parties.ship_to":
        ship_to_rule = party
        print(f"Ship To rule: {ship_to_rule}")
    elif party['mapTo'] == "parties.bill_to":
        bill_to_rule = party
        print(f"Bill To rule: {bill_to_rule}")

Ship From rule: {'seg': 'N1', 'qual': 'SH', 'nameIdx': 2, 'mapTo': 'parties.ship_from'}
Ship To rule: {'seg': 'N1', 'qual': 'CN', 'nameIdx': 2, 'mapTo': 'parties.ship_to'}
Bill To rule: {'seg': 'N1', 'qual': 'BT', 'nameIdx': 2, 'mapTo': 'parties.bill_to'}


In [9]:
for date in dates:
    if date['mapTo'] == "dates.pickup":
        pickup_date_rule = date
        print(f"Ship Date rule: {pickup_date_rule}")
    elif date['mapTo'] == "dates.delivery":
        delivery_date_rule = date
        print(f"Delivery Date rule: {delivery_date_rule}")

Ship Date rule: {'seg': 'G62', 'qual': '11', 'mapTo': 'dates.pickup', 'fmt': 'yyyyMMdd'}
Delivery Date rule: {'seg': 'G62', 'qual': '70', 'mapTo': 'dates.delivery', 'fmt': 'yyyyMMdd'}


In [10]:
if charges['strategy'] == "L1_only" or charges['strategy'] == "L1_then_SAC":
    l1_rules = charges['l1_rules']
    sac_rules = None
    print(f"L1 Rules: {l1_rules}")
    if charges['strategy'] == "L1_then_SAC":
        sac_rules = charges['sac_rules']
        print(f"SAC Rules: {sac_rules}")
elif charges['strategy'] == "SAC_only":
    l1_rules = None
    sac_rules = charges['sac_rules']
    print(f"SAC Rules: {sac_rules}")

L1 Rules: [{'contains': ['BASE FREIGHT', 'LINEHAUL'], 'mapTo': 'charges.base_freight'}, {'contains': ['FSC', 'FUEL SURCHARGE'], 'mapTo': 'charges.fuel_surcharge'}, {'contains': ['DETENTION'], 'mapTo': 'charges.detention'}, {'defaultOther': True}]


In [32]:
warinings = []
confidence = 1.0
our_broker = 'OURBROKER'
try:
    invoice_id = segments[invoice_id_rule['seg']][0][invoice_id_rule['idx']].strip()
    print(f"Invoice ID: {invoice_id}")
except KeyError:
    invoice_id = None
    print("Invoice ID not found.")
    confidence = 0.1
    
if our_broker == segments['GS'][0][3].strip():
    side = 'Buy'
else:
    side = 'Sell'
print(f"Side: {side}")
source = { "type": "edi210", "doc_uri": "null"}
print(f"Source: {source}")

Invoice ID: INV1001
Side: Buy
Source: {'type': 'edi210', 'doc_uri': 'null'}


In [12]:
carrier = {"name": "null", "scac": "null"}
print(f"Carrier: {carrier}")
customer = {"name": "null", "account_id": "null"}
print(f"Customer: {customer}")

Carrier: {'name': 'null', 'scac': 'null'}
Customer: {'name': 'null', 'account_id': 'null'}


In [33]:
bol = segments.get(bol_rule[0]['seg'], [])
if len(bol) > 0:
    bol = bol[0][bol_rule[0]['idx']].strip()
else:
    bol = segments.get(bol_rule[1]['seg'], [])
    if len(bol) > 0:
        for item in bol:
            bol = bol[0][bol_rule[1]['idx']].strip()
            if item[1].strip() == bol_rule[1]['qual']:
                bol = bol[0][bol_rule[1]['idx']].strip()
    else:
        bol = "null"
        warinings.append(f"{bol_rule[1]['seg']} not found.")
        confidence -= 0.05
print(f"BOL: {bol}")

BOL: BOL778231


In [34]:
pro = "null"
try:
    for pro_seg in segments[pro_rule['seg']]:
        if pro_seg[1].strip() == pro_rule.get('qual', 'CN'):
            pro = pro_seg[pro_rule['idx']].strip()
except:
    pro = "null"
    warinings.append(f"{pro_rule['seg']} not found.")
    confidence -= 0.05
print(f"PRO: {pro}")

PRO: PR0123456


In [35]:
load_id = "null"
po = "null"
print(f"PO: {po}")
try:
    for load_id_seg in segments[load_id_rule['seg']]:
        if load_id_seg[1].strip() == load_id_rule.get('qual', 'LO'):
            load_id = load_id_seg[pro_rule['idx']].strip()
except:
    load_id = "null"
    warinings.append(f"{load_id_rule['seg']} not found.")
    confidence -= 0.05
print(f"Load ID: {load_id}")

PO: null
Load ID: LD-56712


In [42]:
ship_from = "null"
ship_to = "null"    
bill_to = "null"

if segments.get(parties[0]['seg']) is None:
    warinings.append(f"{parties[0]['seg']} segment not found.")
    confidence -= 0.05

else:
    for N1_seg in segments[parties[0]['seg']]:
        if N1_seg[1] == ship_from_rule.get('qual', 'SH'):
            ship_from = N1_seg[ship_from_rule['nameIdx']].strip()
        elif N1_seg[1] == ship_to_rule.get('qual',"CN"):
            ship_to = N1_seg[ship_to_rule['nameIdx']].strip()
        elif N1_seg[1] == bill_to_rule.get('qual',"BT"):
            bill_to = N1_seg[bill_to_rule['nameIdx']].strip()
print(f"Ship From: {ship_from}")
print(f"Ship To: {ship_to}")
print(f"Bill To: {bill_to}")

Ship From: XYZ CORP
Ship To: ABC DISTRIBUTION
Bill To: OURBROKER AP


In [ ]:
pickup = "null"
delivery = "null"

if segments.get(dates[0]['seg']) is None:
    warinings.append(f"{dates[0]['seg']} segment not found.")
    confidence -= 0.05

else:
    for date in segments[dates[0]['seg']]:
        if date[1] == pickup_date_rule.get('qual',"11"):
            pickup = date[2].strip()
        elif date[1] == delivery_date_rule.get('qual',"70"):
            delivery = date[2].strip()
print(f"Pickup Date: {pickup}")
print(f"Delivery Date: {delivery}") 
try:
    invoice_date = segments[invoice_date_rule['seg']][0][invoice_date_rule['idx']].strip()
    print(f"Invoice Date: {invoice_date}")
except KeyError:
    invoice_date = "null"
    warinings.append(f"{invoice_date_rule['seg']} not found.")
    confidence = 0.1
    print("Invoice Date segment not found.")  

Pickup Date: 20251030
Delivery Date: 20251102
Invoice Date: 20251101


In [ ]:
base_freight = 0.00
fuel_surcharge = 0.00
detention = 0.00
other = []
if l1_rules:
    # print(l1_rules)
    # print(segments['L1'])
    for l1_rule in l1_rules:
        # print(f"L1 Rule: {l1_rule}")
        mapto = l1_rule.get('mapTo','charges.other').split('.')[-1]
        # print(f"MapTo: {mapto}")
        contains = l1_rule.get('contains', [])
        # print(f"Contains: {contains}")
        try:
            for l1_charge in segments['L1']:
                # print(f"L1 Charge: {l1_charge}")
                if any(c in l1_charge for c in contains):
                    if mapto == "base_freight":
                        base_freight += float(l1_charge[2].strip())
                        # print(f"Base Freight updated: {base_freight}")
                    elif mapto == "fuel_surcharge":
                        fuel_surcharge += float(l1_charge[2].strip())
                        # print(f"Fuel Surcharge updated: {fuel_surcharge}")
                    elif mapto == "detention":
                        detention += float(l1_charge[2].strip())
                        # print(f"Detention updated: {detention}")
                    elif mapto == "other":
                        other.append({
                            "code": l1_charge[2].strip(),
                            "desc": l1_charge[-1].strip(),
                            "amount": float(l1_charge[2].strip())
                        })
                        warinings.append(f"Other charge added: {l1_charge[-1].strip()} - {l1_charge[2].strip()}")
                        confidence -= 0.1
        except:
            if charges["strategy"] == "L1_then_SAC" or charges["strategy"] == "SAC_only":
                warinings.append("L1 segment not found.")
            else:
                confidence = 0.1
                raise KeyError("L1 segment not found.")
if sac_rules:
    for sac_rule in sac_rules:
        # print(f"L1 Rule: {l1_rule}")
        mapto = sac_rule.get('mapTo','charges.other').split('.')[-1]
        # print(f"MapTo: {mapto}")
        codeIn = sac_rule.get('codeIn', [])
        # print(f"Contains: {contains}")
        try:
            for sac_charge in segments['SAC']:
                # print(f"L1 Charge: {l1_charge}")
                if any(c in sac_charge for c in codeIn):
                    if mapto == "fuel_surcharge":
                        fuel_surcharge += float(sac_charge[2].strip())
                        # print(f"Base Freight updated: {base_freight}")
                    elif mapto == "detention":
                        detention += float(sac_charge[2].strip())
                        # print(f"Fuel Surcharge updated: {fuel_surcharge}")
                    elif mapto == "other":
                        other.append({
                            "code": sac_charge[2].strip(),
                            "desc": sac_charge[-1].strip(),
                            "amount": float(sac_charge[2].strip())
                        })
                        warinings.append(f"Other charge added: {sac_charge[-1].strip()} - {sac_charge[2].strip()}")
                        confidence -= 0.1
        except:
            if charges["strategy"] == "L1_then_SAC" or charges["strategy"] == "L1_only":
                warinings.append("SAC segment not found.")
            else:
                confidence = 0.1
                raise KeyError("SAC segment not found.")

print(f"Base Freight: {base_freight}")
print(f"Fuel Surcharge: {fuel_surcharge}")
print(f"Detention: {detention}")
print(f"Other Charges: {other}")

Base Freight: 2201.35
Fuel Surcharge: 192.45
Detention: 150.0
Other Charges: []


In [48]:
sum_total = 0.00
sum_total += base_freight
sum_total += fuel_surcharge
sum_total += detention
for oth in other:
    sum_total += oth['amount']
print(f"Sum Total from Charges: {sum_total}")

try:
    total = segments[total_rules['seg']][0][total_rules['idx']].strip()
    print(f"Total from EDI: {total}")
except:
    total = sum_total
    warinings.append("Total segment not found.")
    confidence -= 0.15

if sum_total != float(total):
    warinings.append(f"Total from EDI {total} does not match sum of charges {sum_total}.")
    confidence -= 0.1

Sum Total from Charges: 2543.7999999999997
Total from EDI: 2543.80


In [45]:
try:
    currency = currency_rules['default']
except :
    currency = "USD"
print(f"Currency: {currency}")

Currency: USD


In [46]:
golden_invice = {
    "invoice_id": invoice_id,
    "side": side,
    "source": { "type": "edi210", "doc_uri": 'null' },
    "carrier": { "name": 'null', "scac": 'null' },
    "customer": { "name": 'null', "account_id": 'null' },
    "refs": { "bol": bol, "pro": pro, "po": 'null', "load_id": load_id },
    "parties": { "ship_from": ship_from, "ship_to": ship_to, "bill_to": bill_to },
    "dates": { "invoice": invoice_date, "pickup": pickup, "delivery": delivery },
    "currency": currency,
    "charges": {
      "base_freight": base_freight,
      "fuel_surcharge": fuel_surcharge,
      "detention": detention,
      "other": other
    },
    "total": total,
    "metadata": {
      "golden_schema_version": "0.1",
      "parser_version": "1.0.0",
      "edi_version": edi_version,
      "trading_partner": partner,
      "confidence": confidence
    },
    "evidence": { "doc_uri": 'null', "attachments": [] }
  }
print(json.dumps(golden_invice, indent=2))
print("Warnings:")
for warn in warinings:
    print(f"- {warn}")

{
  "invoice_id": "INV1001",
  "side": "Buy",
  "source": {
    "type": "edi210",
    "doc_uri": "null"
  },
  "carrier": {
    "name": "null",
    "scac": "null"
  },
  "customer": {
    "name": "null",
    "account_id": "null"
  },
  "refs": {
    "bol": "BOL778231",
    "pro": "PR0123456",
    "po": "null",
    "load_id": "LD-56712"
  },
  "parties": {
    "ship_from": "XYZ CORP",
    "ship_to": "ABC DISTRIBUTION",
    "bill_to": "OURBROKER AP"
  },
  "dates": {
    "invoice": "20251101",
    "pickup": "20251030",
    "delivery": "20251102"
  },
  "currency": "USD",
  "charges": {
    "base_freight": 2201.35,
    "fuel_surcharge": 192.45,
    "detention": 150.0,
    "other": []
  },
  "total": "2543.80",
  "metadata": {
    "golden_schema_version": "0.1",
    "parser_version": "1.0.0",
    "edi_version": "004010",
    "trading_partner": "CARRIERX",
    "confidence": 0.9
  },
  "evidence": {
    "doc_uri": "null",
    "attachments": []
  }
}
Warnings:
- Total from EDI 2543.80 does no